# Coarse Grain the data

The raw gSAM output data (given to us by 
Marat Khairoutdinov) can be found on Bridges2 at:

/ocean/projects/ees240018p/gmooers/gsam_data/

To coarse grain to resolution compatible with it go to the directory:

Regridder/

Run the script to regrid the gSAM data:

new_regridder_terrain_info.py

A bash script to run it in the slurm queue is called:

bash.sh

In [ ]:
!python3 new_regridder_terrain_info.py # to run

or

In [ ]:
! sbatch bash.sh

I saved my coarsened simulation to:

/ocean/projects/ees240018p/gmooers/gsam_data/cam_resolution_gsam_v2/ (line 51 in gsam_main_loop.py)

# Interpolate the data from z to sigma

In the SAM_Data_Collection/ directory:

compile advect_scalar3D_f2py.f90 using the 'my_bash_script' bash file

In [ ]:
! f2py3 -c -m  advect_scalar3D_f2py_my_min2  advect_scalar3D_f2py.f90

run the 'copy_mult_from_JY' file to generate .py and bash scripts for all the .nc files in /ocean/projects/ees240018p/gmooers/gsam_data/cam_resolution_gsam_v2/ and then run the generated bash scripts in the queue

In [ ]:
! bash copy_mult_from_JY

In [ ]:
! sbatch my_bash_script

# Preprocess the data

In this stage, our specific inputs and outputs for the neural network are derived from the coarse-grained simulation data created in the two stages above. The output will be a series of .nc files with these input/output variables. Note -- this is just the derivation of the variables -- scaling occurs later.

The essential script can be found in the directory:

NN_Input_Output_Data/input_output_data_preparation.py

The python script will be populated by yaml files. These are found at:

NN_Input_Output_Data/configs/

You can see two different configs:

config_cam_1.yaml 

When populated by one of the config files above, the python script goes through the coarse-grained simulation create the input/output variables. In more detail, here are the flags in a given .yaml file:

An example of how to generate the training and test data is:

In [ ]:
! python3 input_output_data_preparation.py configs/config_cam_1.yaml

However, for any significant data volume, I recommend submitting the job to the slurm que via the bashscripts for training and testing data found in the Bash_Scripts/ directory within NN_Input_Output_Data/

The key parts of the config to update are 

In [ ]:
# where the coarse-grained data is coming from (this will select all 877 .nc files)
filepath: "/ocean/projects/ees240018p/gmooers/Regridding/Regridded_Data/*0000[012]**.nc4"
# where the inputs/outputs will be saved to
savepath: "/ocean/projects/ees240018p/gmooers/Regridding/Training_Data/"
# how many vertical levels in the column to use in the inputs/outputs
levels: 49
# if getting terrain information, what level to take  -- should this be 0?
ground_levels: 1
# identifying name
my_name: "Input_Output_Data_Training_Part_1_"
# if True, get additional information for outputs (std / std_min) for scaling for the NN
rewight_outputs: False

#below are what variables to put in the input and output vector
# Tin = Absolute Temperature of the column at each level in (K)
# Qin = QC (Cloud Water) + QV (Water Vapor) + QI (Cloud Ice) in (g/kg)
# skt = skin temperature at the given atmospheric temperature
# cos_lat = cosine of the latitude at the given atmospheric column
# sfc_pres = surface reference pressure -- proxy for terrain at a given atmospheric column
# land_frac = land or ocean (1 or 0) at a given atmospheric column
# predict tendencies -- set to True to make script create your output vector of fluxes/tends
# grid cell area -- meters squared area of cell on the cam grid

flag_dict:
  Tin_feature: True
  qin_feature: True
  predict_tendencies: True
  skt: False
  land_frac: True
  sfc_pres: True
  cos_lat: False
  grid_cell_area: True

# Running the experiment

When an experiment is launched, three things happen:

1). Final preparations for the data are made \
2). The final form of the training/test data is passed to the neural network and it is trained \
3). Post-processing analysis is conducted to examine the network performance

You can launch an experiment with two things:

1). The python script ml_train_script_netcdf.py (named because it works with prepared data in the form of .nc files) \
2). a .yaml config file specifying what final preparations will be to the data and hyperparameter choices for the neural network

This portion of the workflow occurs in the directory Training/.

An example of an experiment launch from the command line looks like:

In [ ]:
! python3 neural_network_training_Original_JN.py New_Configs/config_10_original_CAM.yaml

With any significant amount of data, it should never be run without a cpu or gpu resource and it is more strategic to submit in a bashscript. You can find an example at

Bash_Scripts/

In [ ]:
! sbatch bashcpu_cam_original_10.sh

However, within the bash script make sure your paths are correct for your:

1). Python environment
2). Github repository

Full example below

In [ ]:
#!/bin/bash

#SBATCH -A ees220005p
#SBATCH --job-name="original_CAM_trial_10"
#SBATCH -o "outputs/original_CAM_trial_10.%j.%N.out"
#SBATCH -p RM-512 #could do RM-512, RM-shared, RM
#SBATCH -N 1
#SBATCH --ntasks-per-node=128
#SBATCH --export=ALL
#SBATCH -t 72:00:00 # max of 48 hours for GPU
#SBATCH --mem=492G
#SBATCH --no-requeue

module purge

source /jet/home/gmooers/miniconda3/bin/activate torchenv

cd /ocean/projects/ees240018p/gmooers/Githubs/Neural_nework_parameterization/NN_training/src/

python3 neural_network_training_Original_JN.py /ocean/projects/ees240018p/gmooers/Githubs/Neural_nework_parameterization/NN_training/run_training/Improved_run_Experiments/Config_Files/New_Configs/config_10_original_CAM.yaml

This will succsessfully launch an experiment. However, for more detail on what is going on during the run, see below

## Part 1: Final Data Manipulation

When an experiment is launched, the first thing that happens is the data is transformed once more. The specifics of this will be dictated by some of the flags in the .yaml file (New_Configs/).

These include:

In [ ]:
# sets the path to the training data
training_expt : "/ocean/projects/ees240018p/gmooers/Regridding/Training_Data/CAM_Trial_Data_TTTTFTF_Train.nc"
# sets the path to the test data
test_expt : '/ocean/projects/ees240018p/gmooers/Regridding/Training_Data/CAM_Trial_Data_TTTTFTF_Test.nc'
# sets the path for extra weighting in the scaling of the variables
weights_path : null # or path
# sets what variables will be in the neural network input vector
input_vert_vars : ['Tin','qin','terra','sfc_pres'] 
# sets what variables will be in the neural network output vector
output_vert_vars : ['Tout', 'T_adv_out','q_adv_out','q_auto_out','q_sed_flux_tot']
# If True, nothing. If False, removes the poles from the training data (70N-90N; 70S-90S)
poles: True
# what percentage of the trainng data to use
training_data_volume: 25.0
# what percentage of the test data to use
test_data_volume: 50.0
# if True, use the weights in the scaling of the data; If False, nothing
rewight_outputs: False

All the above information will be passed into the training script neural_network_training_Original_JN.py, however, the details of the work happen in the LoadDataStandardScaleData_v4() function of data_scalar_and_reshaper_original.py which is called into neural_network_training_Original_JN.py in the beginning.

A simplified version of the logic and steps of LoadDataStandardScaleData_v4() is depicted below:

In [ ]:
# xarray opens the data -- this approach avoids loading into memory (both train + test)
train_store = xr.open_mfdataset(traindata)

# inputs (and afterwards using the same logic outputs) are extracted from the .nc files -- below showing inputs
# training and test following the same logic as well -- but will show training below
for inputs in input_vert_vars:
    # array is of shape (z dimension, sample) a.k.a. (z, lat*time*lon)
    train_dask_array = train_store[inputs]
    # inputs are scaled (x - mean / std) by vertical level except for scalars; outputs are scaled neglecting vertical level
    # see normalize_by_level(); standardize_outputs() helper function for details on implementation
    scaled_train_dask_array, mean, std = normalize_by_level(train_dask_array)
    # data is spliced based on given training/test data percentage you wish to use
    scaled_train_dask_array = scaled_train_dask_array[:, :training_data_percentage]
    # if the input is a scalar (land frac, sfc pres), it is reshaped from (sample) to (1,sample) -- not relevant for outputs
    scaled_train_dask_array = scaled_train_dask_array.expand_dims(dim='new_dim', axis=0)
    # saved to a dictionary
    scaled_inputs['train'][inputs] = scaled_train_dask_array

# after the dictionary of inputs (or outputs) of training (or test) data has been built, change it to a single DataArray Structure
# This DataArray structure is concatenated along the Z axis (axis 0)
# Example dictionary of Tin (49, 1000000), Qin (49,1000000), land_frac (1, 1000000), sfc_pres (1, 1000000) ==> DataArray (100, 1000000)
# see convert_dict_to_array() helper function for implementation details
final_train_inputs = convert_dict_to_array(scaled_inputs['train'])

# swap the axis so the final array is (sample, input number) to work with the pytorch NN
final_train_inputs = final_train_inputs.swapaxes(0,1)

The above does not cover optional flags including special weighting and removal of the poles which are also part of the full LoadDataStandardScaleData_v4() function if desired

## Part 2: Training

Training details are also primarily set in the .yaml config. Relevant flags are:

In [ ]:
# sets the number of hidden layers in the NN and the number of neurons (size) of each layer
layer_sizes: [1024, 512, 256, 128, 64]
# ensures unbiased result by selecting a random seed
random_seed: 42
# number of epochs to train NN
epochs: 7
# the learning rate for the NN
lr: 0.0000001
# NN batch size
batch_size: 1024 
# set True to train a NN; False assumes NN was trained previously
train_new_model: True

Below are some of the key steps in the script.

In [ ]:
# In the script itself the model information is instantiated by the CustomNN() class
# input/output size determined by the outputs of LoadDataStandardScaleData_v4() higher up in code
model = CustomNN(input_size, layer_sizes, output_size)

# machine identifies if CPU or GPU
device = "cuda" if torch.cuda.is_available() else "CPU"
# Use DataParallel for multiple GPUs -- this allows for the option of future paralleization -- not currently leveraged
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = nn.DataParallel(model)

model = model.to(device)

# Creates a diagram of the model structure for visualization
model_graph = draw_graph(model, 
                                input_size=(batch_size, input_size),
                                graph_name=nametag,
                                save_graph=True,
                                directory=path_for_design,
                                    )

# starting point standard loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# model training is handled in function new_train_model()
if train_new_model is True:
    train_losses, test_losses, train_accuracies, test_accuracies, avg_epoch_time = new_train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=epochs, device=device)

    # Plot the losses and accuracies for each epoch
    plot_metrics(train_losses, test_losses, train_accuracies, test_accuracies, avg_epoch_time, name, save_path, nametag)

After successfully passing the training code the model now enters post-processing

In [ ]:
# get predictions for the test data from the trained model
scaled_predictions = get_model_predictions_from_numpy(model=model, #trained model
                                     test_data=test_inputs_scaled, #scaled test data
                                     batch_size=batch_size,
                                                         )
# unscale the predictions to compare to the outputs
unscaled_predictions = undo_scaling(
                                scaled_array=scaled_predictions, 
                                scaler_dict=train_outputs_pp_dict, 
                                vertical_dimension=levels, 
                                output_variables=output_vert_vars,
                                )

# plotting and analysis   
post_processing_figures.main_plotting(truth=np.transpose(test_outputs_original), # put z dimension first for plotting code
                                              pred=np.transpose(unscaled_predictions), #put z dimension first for plotting code 
                                              raw_data=single_file,
                                              z_dim=levels,
                                             var_names=output_vert_vars,
                                              save_path=save_path,
                                              nametag=nametag,
                                             )

Go to the savepath from the config file (example below) andf see how your neural network performed

In [ ]:
save_path: '/ocean/projects/ees240018p/gmooers/Investigations/Model_Performance/'